## Creating SQLite Database

In [1]:
# Import libraries
import pandas as pd
import sqlite3

In [ ]:
# Install the ipthon-sql òlibrary
#!pip install ipython-sql

In [ ]:
# To guarantee compatibility between jupysql / ipython-sql and newer prettytable releases
#!pip install "prettytable<3.10"

In [ ]:
#CANCELLARE !pip install --upgrade ipython-sql prettytable

In [3]:
# Import data
df = pd.read_csv("WorldLifeExpectancy.csv")
# df.head()

In [5]:
# Create a connetion using sqlite3 library
cnn = sqlite3.connect('World_Life_Expectancy.db')

In [7]:
# Add data to the database
df.to_sql('World_Life_Expectancy', cnn)

2941

In [9]:
# Load the sql module to Ipython
%load_ext sql

In [11]:
# Point the jupiter sql library to the database
%sql sqlite:///World_Life_Expectancy.db

# Set a compatible style
%config SqlMagic.style = 'PLAIN_COLUMNS'

# Tell sql to output to a Pandas DataFrame directly
%config SqlMagic.autopandas = True

In [13]:
%%sql

SELECT * 
FROM World_Life_Expectancy

 * sqlite:///World_Life_Expectancy.db
Done.


,index,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,percentage expenditure,Measles,BMI,under-five deaths,Polio,Diphtheria,HIV/AIDS,GDP,thinness 1-19 years,thinness 5-9 years,Schooling,Row_ID
0,0,Afghanistan,2022,Developing,65.0,263,62,71.3,1154,19.1,83,6,65,0.1,584,17.2,17.3,10.1,1
1,1,Afghanistan,2021,Developing,59.9,271,64,73.5,492,18.6,86,58,62,0.1,613,17.5,17.5,10.0,2
2,2,Afghanistan,2020,Developing,59.9,268,66,73.2,430,18.1,89,62,64,0.1,632,17.7,17.7,9.9,3
3,3,Afghanistan,2019,Developing,59.5,272,69,78.2,2787,17.6,93,67,67,0.1,670,17.9,18.0,9.8,4
4,4,Afghanistan,2018,Developing,NaN,275,71,7.1,3013,17.2,97,68,68,0.1,64,18.2,18.2,9.5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2936,2936,Zimbabwe,2011,Developing,44.3,723,27,0.0,31,27.1,42,67,65,33.6,454,9.4,9.4,9.2,2937
2937,2937,Zimbabwe,2010,Developing,44.5,715,26,0.0,998,26.7,41,7,68,36.7,453,9.8,9.9,9.5,2938
2938,2938,Zimbabwe,2009,Developing,44.8,73,25,0.0,304,26.3,40,73,71,39.8,57,1.2,1.3,10.0,2939
2939,2939,Zimbabwe,2008,Developing,45.3,686,25,0.0,529,25.9,39,76,75,42.1,549,1.6,1.7,9.8,2940


## Data Cleaning

The aim of data cleaning is to enhance the quality and reliability of a dataset by detecting and correcting errors or inconsistencies. The overall goal is to prepare the data so it can be trusted and effectively used for analysis, modeling, or decision-making. Examples include handling duplicate records, missing values, or invalid entries such as zeros where they are not appropriate.

In [16]:
%%sql

-- Count all rows

SELECT COUNT(*) AS total_rows
FROM World_Life_Expectancy

 * sqlite:///World_Life_Expectancy.db
Done.


,total_rows
0,2941


In [18]:
%%sql

-- Standardize column names

ALTER TABLE World_Life_Expectancy RENAME COLUMN "Life expectancy " TO "Life_expectancy";
ALTER TABLE World_Life_Expectancy RENAME COLUMN "Adult Mortality" TO "Adult_mortality";
ALTER TABLE World_Life_Expectancy RENAME COLUMN "infant deaths" TO "Infant_deaths";
ALTER TABLE World_Life_Expectancy RENAME COLUMN "percentage expenditure" TO "Percentage_expenditure";
ALTER TABLE World_Life_Expectancy RENAME COLUMN " BMI " TO "BMI";
ALTER TABLE World_Life_Expectancy RENAME COLUMN "under-five deaths " TO "Under_five_deaths";
ALTER TABLE World_Life_Expectancy RENAME COLUMN " HIV/AIDS" TO "hiv_aids";
ALTER TABLE World_Life_Expectancy RENAME COLUMN " thinness  1-19 years" TO "Thinness_1_19_years";
ALTER TABLE World_Life_Expectancy RENAME COLUMN " thinness 5-9 years" TO "Thinness_5_9_years";

 * sqlite:///World_Life_Expectancy.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


""


In [20]:
%%sql

-- Country and Year values should be unique

SELECT Country, Year, CONCAT(Country, Year), COUNT(CONCAT(Country, Year)) 
FROM World_Life_Expectancy
GROUP BY Country, Year, CONCAT(Country, Year)
HAVING COUNT(CONCAT(Country, Year)) > 1

 * sqlite:///World_Life_Expectancy.db
Done.


,Country,Year,"CONCAT(Country, Year)","COUNT(CONCAT(Country, Year))"
0,Ireland,2022,Ireland2022,2
1,Senegal,2009,Senegal2009,2
2,Zimbabwe,2019,Zimbabwe2019,2


In [22]:
%%sql

-- Identify the IDs corresponding to the Year + Country combinations that have duplicate values in order to remove them from the data

SELECT *
FROM(
SELECT Row_ID, 
CONCAT(Country, Year),
ROW_NUMBER() OVER(PARTITION BY CONCAT(Country, Year) ORDER BY CONCAT(Country, Year)) as Row_Num
FROM World_Life_Expectancy
) AS Row_Table
WHERE Row_Num > 1 

 * sqlite:///World_Life_Expectancy.db
Done.


,Row_ID,"CONCAT(Country, Year)",Row_Num
0,1252,Ireland2022,2
1,2265,Senegal2009,2
2,2929,Zimbabwe2019,2


In [24]:
%%sql

-- Remove IDs from the table

DELETE FROM World_Life_Expectancy
WHERE Row_ID IN (
	SELECT Row_ID
	FROM(
	SELECT Row_ID, 
	CONCAT(Country, Year),
	ROW_NUMBER() OVER(PARTITION BY CONCAT(Country, Year) ORDER BY CONCAT(Country, Year)) as Row_Num
	FROM World_Life_Expectancy
	) AS Row_Table
	WHERE Row_Num > 1 
)

 * sqlite:///World_Life_Expectancy.db
3 rows affected.


""


In [26]:
%%sql

-- Check for any missing or null values in the Status column

SELECT * 
FROM World_Life_Expectancy
WHERE Status IS NULL

 * sqlite:///World_Life_Expectancy.db
Done.


,index,Country,Year,Status,Life_expectancy,Adult_mortality,Infant_deaths,Percentage_expenditure,Measles,BMI,Under_five_deaths,Polio,Diphtheria,hiv_aids,GDP,Thinness_1_19_years,Thinness_5_9_years,Schooling,Row_ID
0,8,Afghanistan,2014,None,57.5,295,82,10.9,1141,15.2,113,63,63,0.1,370,19.0,19.1,8.4,9
1,17,Albania,2021,None,77.5,8,0,428.7,0,57.2,1,98,98,0.1,4576,1.2,1.3,14.2,18
2,988,Georgia,2012,None,73.9,128,1,9.4,1356,48.6,1,82,82,0.1,154,2.8,2.9,12.2,989
3,990,Georgia,2010,None,72.7,132,1,70.5,216,47.5,2,74,75,0.1,928,2.9,3.0,11.8,991
4,2797,United States of America,2021,None,79.1,14,23,0.0,667,69.1,27,93,95,0.1,0,0.8,0.6,0.0,2798
5,2846,Vanuatu,2020,None,71.6,135,0,447.5,0,51.7,0,65,64,0.1,3167,1.5,1.4,10.8,2847
6,2914,Zambia,2016,None,57.4,368,30,143.9,26,2.2,47,93,94,9.1,1139,6.7,6.6,11.6,2915
7,2918,Zambia,2012,None,49.3,554,34,121.9,45,18.4,55,84,82,17.0,691,7.1,7.0,10.7,2919


In [28]:
%%sql

-- Check for non-null values in the status column.

SELECT DISTINCT(Status) 
FROM World_Life_Expectancy
WHERE Status <> ''

 * sqlite:///World_Life_Expectancy.db
Done.


,Status
0,Developing
1,Developed


In [32]:
%%sql

-- Update the values of the rows where the status is null, based on the status information recorded in other rows for the same country

UPDATE World_Life_Expectancy
SET Status = 'Developing'
WHERE Status IS NULL
  AND Country IN (
      SELECT Country
      FROM World_Life_Expectancy
      WHERE Status = 'Developing'
  )
;

UPDATE World_Life_Expectancy
SET Status = 'Developed'
WHERE Status IS NULL
  AND Country IN (
      SELECT Country
      FROM World_Life_Expectancy
      WHERE Status = 'Developed'
  )

 * sqlite:///World_Life_Expectancy.db
7 rows affected.
1 rows affected.


""


In [34]:
%%sql

-- Check for any missing or null values in the Life Expectancy column

SELECT * 
FROM World_Life_Expectancy
WHERE Life_expectancy IS NULL

 * sqlite:///World_Life_Expectancy.db
Done.


,index,Country,Year,Status,Life_expectancy,Adult_mortality,Infant_deaths,Percentage_expenditure,Measles,BMI,Under_five_deaths,Polio,Diphtheria,hiv_aids,GDP,Thinness_1_19_years,Thinness_5_9_years,Schooling,Row_ID
0,4,Afghanistan,2018,Developing,None,275,71,7.1,3013,17.2,97,68,68,0.1,64,18.2,18.2,9.5,5
1,20,Albania,2018,Developing,None,88,0,437.1,28,55.1,1,99,99,0.1,4437,1.4,1.5,13.3,21


In [36]:
%%sql

-- For Afghanistan and Albania, we can see a gradual increase in life expectancy over the years, so we can replace the null value with 
-- the mean average of the values from the previous and next year 

SELECT * 
FROM World_Life_Expectancy
WHERE Country IN ('Afghanistan', 'Albania')

 * sqlite:///World_Life_Expectancy.db
Done.


,index,Country,Year,Status,Life_expectancy,Adult_mortality,Infant_deaths,Percentage_expenditure,Measles,BMI,Under_five_deaths,Polio,Diphtheria,hiv_aids,GDP,Thinness_1_19_years,Thinness_5_9_years,Schooling,Row_ID
0,0,Afghanistan,2022,Developing,65.0,263,62,71.3,1154,19.1,83,6,65,0.1,584,17.2,17.3,10.1,1
1,1,Afghanistan,2021,Developing,59.9,271,64,73.5,492,18.6,86,58,62,0.1,613,17.5,17.5,10.0,2
2,2,Afghanistan,2020,Developing,59.9,268,66,73.2,430,18.1,89,62,64,0.1,632,17.7,17.7,9.9,3
3,3,Afghanistan,2019,Developing,59.5,272,69,78.2,2787,17.6,93,67,67,0.1,670,17.9,18.0,9.8,4
4,4,Afghanistan,2018,Developing,NaN,275,71,7.1,3013,17.2,97,68,68,0.1,64,18.2,18.2,9.5,5
5,5,Afghanistan,2017,Developing,58.8,279,74,79.7,1989,16.7,102,66,66,0.1,553,18.4,18.4,9.2,6
6,6,Afghanistan,2016,Developing,58.6,281,77,56.8,2861,16.2,106,63,63,0.1,446,18.6,18.7,8.9,7
7,7,Afghanistan,2015,Developing,58.1,287,80,25.9,1599,15.7,110,64,64,0.1,373,18.8,18.9,8.7,8
8,8,Afghanistan,2014,Developing,57.5,295,82,10.9,1141,15.2,113,63,63,0.1,370,19.0,19.1,8.4,9
9,9,Afghanistan,2013,Developing,57.3,295,84,17.2,1990,14.7,116,58,58,0.1,273,19.2,19.3,8.1,10


In [38]:
%%sql

-- For Afghanistan, we can see a gradual increase in life expectancy over the years, so we can replace the null value with 
-- the mean average of the values from the previous and next year 

SELECT t1.Country, t1.Year,  t1.Life_expectancy, 
	t2.Country, t2.Year,  t2.Life_expectancy,
    t3.Country, t3.Year,  t3.Life_expectancy,
    ROUND((t2.Life_expectancy + t3.Life_expectancy)/2,1)
FROM World_Life_Expectancy t1
JOIN World_Life_Expectancy t2
	ON t1.Country = t2.Country
    AND t1.Year = t2.Year - 1
JOIN World_Life_Expectancy t3
	ON t1.Country = t3.Country
    AND t1.Year = t3.Year + 1
WHERE t1.Life_expectancy IS NULL     

 * sqlite:///World_Life_Expectancy.db
Done.


,Country,Year,Life_expectancy,Country,Year,Life_expectancy,Country,Year,Life_expectancy,"ROUND((t2.Life_expectancy + t3.Life_expectancy)/2,1)"
0,Afghanistan,2018,None,Afghanistan,2019,59.5,Afghanistan,2017,58.8,59.1
1,Albania,2018,None,Albania,2019,76.9,Albania,2017,76.2,76.6


In [40]:
%%sql

-- Update the table with the new averaged values.

UPDATE World_Life_Expectancy AS w
SET Life_expectancy = (
  SELECT ROUND((p.Life_expectancy + n.Life_expectancy)/2.0, 1)
  FROM World_Life_Expectancy AS p
  JOIN World_Life_Expectancy AS n
    ON p.Country = n.Country
   AND p.Year   = w.Year - 1
   AND n.Year   = w.Year + 1
  WHERE p.Country = w.Country
)
WHERE w.Life_expectancy IS NULL
  AND EXISTS (
    SELECT 1
    FROM World_Life_Expectancy AS p
    JOIN World_Life_Expectancy AS n
      ON p.Country = n.Country
     AND p.Year   = w.Year - 1
     AND n.Year   = w.Year + 1
    WHERE p.Country = w.Country
)

 * sqlite:///World_Life_Expectancy.db
2 rows affected.


""
